In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
class ChatState(TypedDict):
    history: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)

def chat_node(state: ChatState):

    messages = state['history']
    response = llm.invoke(messages)
    return {"history": [response]}

In [ ]:
checkpoint = MemorySaver()


graph = StateGraph(ChatState)

#add node
graph.add_node('chat_node', chat_node)

#add edges
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

workflow = graph.compile(checkpointer=checkpoint)

In [ ]:
thread_id = "1"
while True:
    user_input = input("User: ")
    if user_input.strip().lower() in ["exit", "quit", "bye"]:
        print(f"User: {user_input}\nAI: Signing off. Have a nice time!")
        break

    initial_state = {"history": [HumanMessage(content=user_input)]}

    config = {'configurable' : {"thread_id": thread_id}}
    result = workflow.invoke(initial_state, config=config)
    response_message = result['history'][-1].content
    print(f"User: {user_input}\nAI: {response_message}")